In [1]:
import copy
import os

import sys
mindnlp_path = '/home/ma-user/work/mindnlp'
sys.path.insert(0, mindnlp_path)

# ignore bnb warnings
os.environ["BITSANDBYTES_NOWELCOME"] = "1"

if "RANK_TABLE_FILE" in os.environ:
    del os.environ["RANK_TABLE_FILE"]

In [2]:
import mindspore as ms
from mindspore.common.api import _no_grad
from mindnlp.core import nn, ops
import mindnlp.core.nn.functional as F
from mindnlp import peft

/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/ma-user/anaconda3/envs/MindSpore/lib

In [3]:
ms.manual_seed(0)
X = ops.rand((1000, 20))
y = (X.sum(1) > 10).long()

[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB


In [4]:
n_train = 800
batch_size = 64

In [5]:
X_train = ms.Tensor(X[:n_train])
y_train = ms.Tensor(y[:n_train])
train_dataset = ms.dataset.NumpySlicesDataset((X_train.asnumpy(), y_train.asnumpy()), column_names=["input_ids", "labels"], shuffle=True)
train_dataset = train_dataset.batch(batch_size)

X_eval = ms.Tensor(X[n_train:])
y_eval = ms.Tensor(y[n_train:])
eval_dataset = ms.dataset.NumpySlicesDataset((X_eval.asnumpy(), y_eval.asnumpy()), column_names=["input_ids", "labels"], shuffle=False)
eval_dataset = eval_dataset.batch(batch_size)

In [6]:
print(next(train_dataset.create_dict_iterator()))

{'input_ids': Tensor(shape=[64, 20], dtype=Float32, value=
[[ 3.77188087e-01,  8.24316382e-01,  4.50266242e-01 ...  6.32127762e-01,  6.74743891e-01,  1.32882118e-01],
 [ 9.95139122e-01,  4.34024334e-01,  3.70879769e-01 ...  5.21495342e-01,  3.37470055e-01,  5.44637918e-01],
 [ 8.67708206e-01,  7.94372559e-02,  3.24187040e-01 ...  9.50439095e-01,  4.65711355e-02,  4.17159677e-01],
 ...
 [ 2.53180861e-01,  6.77524805e-02,  1.47833586e-01 ...  8.56523991e-01,  9.86489296e-01,  9.16864157e-01],
 [ 1.45423532e-01,  6.23252988e-01,  5.57705283e-01 ...  7.54238605e-01,  4.58894372e-01,  9.53555703e-01],
 [ 6.45794153e-01,  8.09241533e-02,  7.59914637e-01 ...  3.06894064e-01,  9.91657138e-01,  3.03503990e-01]]), 'labels': Tensor(shape=[64], dtype=Int64, value= [0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 
 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 
 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0])}


In [7]:
print(next(eval_dataset.create_dict_iterator()))

{'input_ids': Tensor(shape=[64, 20], dtype=Float32, value=
[[ 6.48771405e-01,  8.35347295e-01,  4.01633143e-01 ...  4.59447384e-01,  3.98185611e-01,  3.31850052e-01],
 [ 7.09608078e-01,  3.88867140e-01,  7.47578621e-01 ...  5.11571288e-01,  5.43426037e-01,  7.25902319e-01],
 [ 1.60737991e-01,  2.98117518e-01,  6.46145701e-01 ...  3.76462460e-01,  7.21513033e-01,  1.03136778e-01],
 ...
 [ 6.99411273e-01,  1.79358125e-01,  3.13022852e-01 ...  2.06934214e-01,  2.20992446e-01,  5.08313298e-01],
 [ 2.60040879e-01,  3.01995873e-01,  2.88284898e-01 ...  5.76413870e-01,  5.18678308e-01,  7.90544510e-01],
 [ 4.26524282e-01,  7.05174565e-01,  7.25395799e-01 ...  4.95667338e-01,  6.45046115e-01,  2.79562831e-01]]), 'labels': Tensor(shape=[64], dtype=Int64, value= [0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 
 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 
 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1])}


In [8]:
class MLP(nn.Module):
    def __init__(self, num_units_hidden=2000):
        super().__init__()
        self.seq = nn.Sequential(
            nn.Linear(20, num_units_hidden),
            nn.ReLU(),
            nn.Linear(num_units_hidden, num_units_hidden),
            nn.ReLU(),
            nn.Linear(num_units_hidden, 2),
            nn.LogSoftmax(dim=-1),
        )

    def forward(self, X):
        return self.seq(X)

In [9]:
lr = 0.002
batch_size = 64
num_epochs = 30

In [10]:
[(n, type(m)) for n, m in MLP().named_modules()]

[('', __main__.MLP),
 ('seq', mindnlp.core.nn.modules.container.Sequential),
 ('seq.0', mindnlp.core.nn.modules.linear.Linear),
 ('seq.1', mindnlp.core.nn.modules.activation.ReLU),
 ('seq.2', mindnlp.core.nn.modules.linear.Linear),
 ('seq.3', mindnlp.core.nn.modules.activation.ReLU),
 ('seq.4', mindnlp.core.nn.modules.linear.Linear),
 ('seq.5', mindnlp.core.nn.modules.activation.LogSoftmax)]

In [11]:
config = peft.LoraConfig(
    r=8,
    target_modules=["seq.0", "seq.2"],
    modules_to_save=["seq.4"], # ！！
)

In [12]:
from mindnlp.core import optim
from mindnlp.transformers.optimization import get_linear_schedule_with_warmup

module = MLP()
module_copy = copy.deepcopy(module)  # we keep a copy of the original model for later
for name, param in module.named_parameters():
    print(f"{name}: {param.shape}")

seq.0.weight: (2000, 20)
seq.0.bias: (2000,)
seq.2.weight: (2000, 2000)
seq.2.bias: (2000,)
seq.4.weight: (2, 2000)
seq.4.bias: (2,)


In [13]:
model = peft.get_peft_model(module, config)
for name, param in model.named_parameters():
    print(f"{name}: {param.shape}")

AttributeError: 'Sequential' object has no attribute '_cells'

In [14]:
optimizer = optim.Adam([param for name, param in model.named_parameters() if "lora" in name], lr=lr)
criterion = nn.CrossEntropyLoss()
print(optimizer)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataset) * num_epochs),
)

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.002
    maximize: False
    weight_decay: 0
)


In [14]:
model.print_trainable_parameters()

NameError: name 'model' is not defined

In [15]:
[(n, type(m)) for n, m in model.named_modules()]

NameError: name 'model' is not defined

In [17]:
from tqdm import tqdm

from sys import exit
def forward_fn(**batch):
    outputs = model(batch["input_ids"])
    loss = criterion(outputs, batch["labels"])
    # print(loss)
    return loss

grad_fn = ms.value_and_grad(forward_fn, None, model.trainable_params())

def train_step(**batch):
    loss, grads = grad_fn(**batch)
    # print(loss, grads)
    optimizer.step(grads)
    return loss

for epoch in range(num_epochs):
    model.set_train(True)
    train_loss = 0
    train_total_size = train_dataset.get_dataset_size()
    print(f'start epoch: {epoch}')
    for step, batch in enumerate(tqdm(train_dataset.create_dict_iterator(), total=train_total_size)):
        # print(batch)
        # continue
        loss = train_step(**batch)
        train_loss += loss.float()
        lr_scheduler.step()

    # print(f'eval epoch: {epoch}')
    model.set_train(False)
    eval_loss = 0
    eval_total_size = eval_dataset.get_dataset_size()
    for step, batch in enumerate(tqdm(eval_dataset.create_dict_iterator(), total=eval_total_size)):
        with ms._no_grad():
            outputs = model(batch["input_ids"])
        loss = criterion(outputs, batch["labels"])
        eval_loss += loss.float()
    # print(f'finish epoch: {epoch}')
    

    eval_loss_total = (eval_loss / len(eval_dataset)).item()
    train_loss_total = (train_loss / len(train_dataset)).item()
    print(f"{epoch=:<2}  {train_loss_total=:.4f}  {eval_loss_total=:.4f}")

start epoch: 0


100%|██████████| 4/4 [00:00<00:00, 118.19it/s]


epoch=0   train_loss_total=0.6808  eval_loss_total=0.6981
start epoch: 1


100%|██████████| 4/4 [00:00<00:00, 199.44it/s]


epoch=1   train_loss_total=0.6624  eval_loss_total=0.6650
start epoch: 2


...

epoch=27  train_loss_total=0.0173  eval_loss_total=0.1633
start epoch: 28


100%|██████████| 4/4 [00:00<00:00, 189.94it/s]


epoch=28  train_loss_total=0.0172  eval_loss_total=0.1625
start epoch: 29


100%|██████████| 4/4 [00:00<00:00, 185.36it/s]

epoch=29  train_loss_total=0.0169  eval_loss_total=0.1603


In [18]:
for name, param in model.named_parameters():
    print(f"Parameter: {name} \t| Requires Grad: {param.requires_grad}")


Parameter: base_model.model.seq.0.base_layer.weight 	| Requires Grad: False
Parameter: base_model.model.seq.0.base_layer.bias 	| Requires Grad: False
Parameter: base_model.model.seq.0.lora_A.default.weight 	| Requires Grad: True
Parameter: base_model.model.seq.0.lora_B.default.weight 	| Requires Grad: True
Parameter: base_model.model.seq.2.base_layer.weight 	| Requires Grad: False
Parameter: base_model.model.seq.2.base_layer.bias 	| Requires Grad: False
Parameter: base_model.model.seq.2.lora_A.default.weight 	| Requires Grad: True
Parameter: base_model.model.seq.2.lora_B.default.weight 	| Requires Grad: True
Parameter: base_model.model.seq.4.weight 	| Requires Grad: False
Parameter: base_model.model.seq.4.bias 	| Requires Grad: False


In [20]:
for name, param in model.base_model.named_parameters():
    if "lora" not in name:
        continue

    print(f"New parameter {name:<13} | {param.numel():>5} parameters | updated")

New parameter model.seq.0.lora_A.default.weight |   160 parameters | updated
New parameter model.seq.0.lora_B.default.weight | 16000 parameters | updated
New parameter model.seq.2.lora_A.default.weight | 16000 parameters | updated
New parameter model.seq.2.lora_B.default.weight | 16000 parameters | updated
